In [1]:
import time
import torch
from cudaext.ops.trilinear_interpolate.trilinear_interpolate_utils import *
N = 65536
F = 256
rand = torch.rand(N, 8, F, device='cuda')
feats = rand.clone().requires_grad_()
feats2 = rand.clone().requires_grad_()
points = torch.rand(N, 3, device='cuda')*2-1

t = time.time()
out_cuda = Trilinear_interpolation_cuda.apply(feats2, points)
torch.cuda.synchronize()
print('   cuda fw time', time.time()-t, 's')

t = time.time()
out_py = trilinear_interpolation_cpu(feats, points)
torch.cuda.synchronize()
print('pytorch fw time', time.time()-t, 's')

print('fw all close', torch.allclose(out_py, out_cuda))

t = time.time()
loss2 = out_cuda.sum()
loss2.backward()
torch.cuda.synchronize()
print('   cuda bw time', time.time()-t, 's')

t = time.time()
loss = out_py.sum()
loss.backward()
torch.cuda.synchronize()
print('pytorch bw time', time.time()-t, 's')

print('bw all close', torch.allclose(feats.grad, feats2.grad))

   cuda fw time 0.04561972618103027 s
pytorch fw time 0.2103290557861328 s
fw all close True
   cuda bw time 0.08999824523925781 s
pytorch bw time 1.7275352478027344 s
bw all close True
